In [91]:
import pymongo
import numpy as np
import pandas as pd
import os
import datetime
from scipy.stats import zscore
client = pymongo.MongoClient()

In [94]:
db = client['Fields']
field_list = db.list_collection_names()
index = client['Index']

#### 要製作乾淨的feature清單，要單獨處理，依據行業別分開嘗試看看，每個行業的資料都在業內去跟其他相比，最後得到一個由0出發往上下的標準值(standard normalization)
1. 先抓短區間資料嘗試調整資料
2. Na值、inf值、錯誤資料的處理要搞清楚
    * 方法: 看na值比率、極端值差多少之類的，要歸納一個方法出來檢測


# 定義每一個Field
1. 可否fillna(0), 或者是需要fillna(ffill)
2. 極端值處理方法
3. 是否需要除以市值來比較 -- 所有資料都要成為相對資料而非絕對資料
4. 是否需要將非日資料轉為日資料儲存

In [148]:
class Definition_Fields:
    def __init__(self, client: pymongo.MongoClient):
            self.client = client
            self.db = self.client['Fields']
            self.index = self.client['Index']
            self.factor = self.client['Factors']
            self.sector = self.get_sector()
            self.book = pd.DataFrame()
            self.price = pd.DataFrame()


    def get_sector(self) -> dict:
        sector = pd.DataFrame(index['產業名稱'].find({}, {'_id': 0}))
        sector.loc[:, '股票代號'] = sector['股票代號'].astype(str)
        sector_dict = {}
        for s in sector['產業名稱'].unique():
            sector_dict[s] = sector[sector['產業名稱'] == s]['股票代號'].values
        return sector_dict

    def get_fields(self, 
                    field: str = '收盤價', 
                    start: datetime.datetime=datetime.datetime(2020, 1, 1), 
                    end: datetime.datetime=datetime.datetime.today()
        ) -> pd.DataFrame:
        try:
            data = pd.DataFrame(db[field].find({'日期': {'$gte': start, '$lte': end}}, {'_id': 0})).set_index('日期')
        except:
            print(field, ' Error')
        return data

    def creat_to_daily(self, data: pd.DataFrame) -> pd.DataFrame:
        self.price = self.get_fields('收盤價', start=data.index[0], end=data.index[-1])
        n = pd.DataFrame(columns=self.price.columns, index=self.price.index)
        for i in data.index:
            for c in data.columns:
                n.at[i, str(c)] = data.at[i, c]
        return n.fillna(method='ffill').loc[self.price.index, self.price.columns]

    def divide_book(self, data: pd.DataFrame) -> pd.DataFrame:
        self.book = self.get_fields('股本(百萬)', start=data.index[0], end=data.index[-1])
        assert self.book.shape == data.shape, f'The book did not have same shape {self.book.shape} {data.shape}'
        data = data / self.book.loc[data.index, data.columns]
        return data

    def divide_price(self, data: pd.DataFrame) -> pd.DataFrame:
        if len(self.price) == 0:
            self.price = self.get_fields('收盤價', start=data.index[0], end=data.index[-1])
        else:
            assert self.price.shape == data.shape, f'The price did not have same shape {self.price.shape} {data.shape}'
            data = data / self.price.loc[data.index, data.columns]
        return data

    def split_with_sector(self, data: pd.DataFrame) -> dict:
        field_split = {}
        for k, v in self.sector.items():
            field_split[k] = data.loc[:, v]
        return field_split

    def clean_outlier(self, data_s: dict, q: float = 0.01) -> dict:
        for k, v in data_s.items():
            data_s[k].mask(v.ge(v.quantile(1. - q, axis=1)), v.quantile(1. - q, axis=1), inplace=True, axis=1)
            data_s[k].mask(v.le(v.quantile(q , axis=1)), v.quantile(q, axis=1), inplace=True, axis=1)
        return data_s

    def transfer_zscore(self, data_s) -> dict:
        for k, v in data_s.items():
            data_s[k] = v.apply(zscore, axis=1, nan_policy='omit')
            data_s[k].mask(data_s[k] > 5, 5)
            data_s[k].mask(data_s[k] < -5, -5)
            data_s[k].fillna(0, inplace=True)
        return data_s

    def save_db(self, field: str, data_s: dict) -> None:
        """
        field: db_collection name
        data: pd.DataFrame 需要已整理好可使用的資料
        """
        if field in self.factor.list_collection_names():
            print(f'Warming!! DB Had it! {field}')
        else:
            for k, v in data_s.items():
                date = pd.Series(v.index, index=v.index)
                date.name = '日期'
                sec = pd.Series([k] * len(v), name='產業名稱', index=date)
                v = pd.concat([v, date], axis=1)
                v = pd.concat([v, sec], axis=1)
                self.factor[field].insert_many(v.to_dict('records'))
            print(f'Saving {field} are Completed!')
       
    def load_sector(self,
                    field: str, 
                    sector: str,
                    start: datetime.datetime = datetime.datetime(2020, 1, 1), 
                    end: datetime.datetime = datetime.datetime.today()
        ) -> pd.DataFrame:
        return pd.DataFrame(self.factor[field].find({'產業名稱': sector, '日期': {'$gte': start, '$lte': end}}, {'_id': 0}))


    def preprocessing(self, fields: list, 
                    start: datetime.datetime=datetime.datetime(2020, 1, 1), 
                    end: datetime.datetime=datetime.datetime.today(),
                    to_daily: bool=False, b: bool=False, p: bool=False, fillna: bool=True, ffill: bool=False, out: bool=True, z: bool=True, q: float=0.01, save: bool=True
        ) -> pd.DataFrame:
        for field in fields:
            """
            1. 可以fillna(0)
            2. 直接跟其他比較
            3. 用1%與99%避免奇怪極端值
            """
            # 步驟0. 確認db沒資料(節省時間)
            if field in self.factor.list_collection_names():
                print(f'Warming!! DB Had it! {field}')
                continue
            # 步驟1. 取得資料
            data = self.get_fields(field, start=start, end=end)
            # 步驟1.5. 除以股本平衡比率 / 除以價格計算比率
            if to_daily:
                data = self.creat_to_daily(data)
            if b:
                data = self.divide_book(data)
            if p:
                data = self.divide_price(data)
            # 步驟2. 處理缺漏值

            if fillna:
                data.fillna(0, inplace=True)
            if ffill:
                data.fillna(method='ffill', inplace=True)

            # 步驟3. 分到各sector
            data_s = self.split_with_sector(data)
            # 步驟4. 根據各sector內容，處理outlier
            if out:
                data_s = self.clean_outlier(data_s, q=q)
            else:
                print(f'Cause not ues outlier, Please check these data are all right.\nField: {field} Max: {data.max().max()} Min: {data.min().min()}')
            # 步驟5. 計分，目前有zscore
            if z:
                data_s = self.transfer_zscore(data_s)
            # 步驟6
            if save:
                self.save_db(field, data_s)


##### 將Collections分類

In [149]:
DF = Definition_Fields(client)
# 不含金額類 23個
"""
籌碼數量類:
    fillna=True, out=False, z=True, save: bool=True)
"""
籌碼數量類 = [
    '主力券商淨買超家數',
    '主力券商淨賣超家數',
    '主力買賣超(張數)',
    '券增減',
    '券現償',
    '券買',
    '券賣',
    '券餘',
    '外資買賣超',
    '投信買賣超',
    '自營商買賣超',
    '自營商買賣超(自行買賣)',
    '自營商買賣超(避險)',
    '買超券商家數',
    '賣超券商家數',
    '資券相抵',
    '資增減',
    '資現償',
    '資買',
    '資賣',
    '資限',
    '資餘',
    '外資及陸資(不含外資自營商)買賣超'
]
DF.preprocessing(籌碼數量類, fillna=True, out=False, z=True, save=True)

# 4個
籌碼金額類 = [
    '券賣金額(千)',
    '外資買賣超金額(千)',
    '投信買賣超金額(千)',
    '自營商買賣超金額(千)',
]
DF.preprocessing(籌碼金額類, fillna=True, out=False, z=True, save=True)

# 10個
籌碼比率類 = [
    '券使用率',
    '券資比',
    '外資尚可投資比率(%)', # 可能有上限100%
    '外資持股比率(%)',
    '投信持股比率(%)',
    '自營商持股比率(%)',
    '融券維持率(%)',
    '融資維持率(%)',
    '整體維持率(%)',
    '資使用率'
]
DF.preprocessing(籌碼比率類, fillna=True, out=True, z=False, save=True)

# 5個
籌碼價格類 = [
    '外資持股成本',
    '投信持股成本',
    '自營商持股成本',
    '融券成本(推估)',
    '融資成本(推估)'
]
DF.preprocessing(籌碼比率類, fillna=False, p=True, out=True, z=False, save=True)

# 13個 s56
基本面_ = [
    'EBITDA_Q',
    '不動產廠房設備_Q',
    '存貨_Q',
    '投資活動現金流量(千)_Q',
    '折舊費用_Q',
    '營業收入淨額_Q',
    '現金及約當現金_Q',
    '現金股利殖利率_Q',
    '負債總計_Q',
    '資產總計_Q',
    '營業收入淨額_Y',
]
DF.preprocessing(基本面_, fillna=False, ffill=True, to_daily=True, b=True, out=True, z=True, save=True)

基本面_1 = [
    'EPS_Q',
    '存貨週轉天數_Q',
    '預估EPS_Q'
    '總資產週轉次數_Y',
]
DF.preprocessing(基本面_1, fillna=False, ffill=True, to_daily=True, out=True, z=True, save=True)

基本面_2 = [
    '槓桿比率_Q',
    '現金流量比率_Q',
    '槓桿比率_Y',
    '現金流量比率_Y',
    '現金股利殖利率_Y',
]
DF.preprocessing(基本面_2, fillna=False, ffill=True, to_daily=True, out=True, z=False, save=True)

# 7個 s68
趴數類 = [
    '均張變動(%)',
    '市值比重(%)',
    '成交值比重(%)',
    '成交量變動(%)',
    '振幅(%)',
    '漲幅(%)',
    '週轉率(%)'
]
DF.preprocessing(趴數類, fillna=True, out=True, z=False, save=True)
# 4個 72
比率類 = [
    '本益比',
    '本益比(近四季)',
    '殖利率',
    '股價淨值比',
]
DF.preprocessing(趴數類, fillna=True, out=True, z=True, save=True)
# 8個 80
價格類 = [
    '開盤價',
    '最高價',
    '最低價',
    '收盤價',
    '還原開盤價',
    '還原最高價',
    '還原最低價',
    '還原收盤價',
]
DF.preprocessing(價格類, fillna=False, ffill=True, p=True, out=False, z=False, save=True)
# 8個 88
其他 = [
    '均張',
    '成交筆數',
    '成交量',
    '成交金額(千)',
    '漲跌',
    '漲跌停',
    '總市值(億)',
    '股本(百萬)',
]
DF.preprocessing(其他, fillna=True, ffill=True, p=False, out=False, z=True, save=True)
type_list = [籌碼數量類, 籌碼金額類, 籌碼比率類, 籌碼價格類, 基本面_, 基本面_1, 基本面_2, 趴數類, 比率類, 價格類, 其他]

Cause not ues outlier, Please check these data are all right.
Field: 主力券商淨買超家數 Max: 20.0 Min: 0.0
Saving 主力券商淨買超家數 are Completed!
Cause not ues outlier, Please check these data are all right.
Field: 主力券商淨賣超家數 Max: 19.0 Min: 0.0
Saving 主力券商淨賣超家數 are Completed!
Cause not ues outlier, Please check these data are all right.
Field: 主力買賣超(張數) Max: 147123.0 Min: -97276.0
Saving 主力買賣超(張數) are Completed!
Cause not ues outlier, Please check these data are all right.
Field: 券增減 Max: 37907.0 Min: -50638.0
Saving 券增減 are Completed!
Cause not ues outlier, Please check these data are all right.
Field: 券現償 Max: 44066.0 Min: 0.0
Saving 券現償 are Completed!
Cause not ues outlier, Please check these data are all right.
Field: 券買 Max: 36408.0 Min: 0.0


提問：如何把單一Fields根據Sections分開?  
A: 先用Section分類股票代號，用loc股票代號分割資料